<a href="https://colab.research.google.com/github/1Kote/reconhecimento_facial_IA/blob/daniel/reconhecimento_facial_EigenFaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reconhecimento facial - Estudo e Trabalho do Dimmy

Bibliotecas para serem importadas

In [ ]:
import cv2, zipfile, os
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow

In [ ]:
#extrair o zip com o dataset
path = '/content/yalefaces.zip'
zip_object = zipfile.ZipFile(file = path, mode='r')
zip_object.extractall('./')
zip_object.close

In [ ]:
print(os.listdir('/content/yalefaces/train'))

In [ ]:
test_image = '/content/yalefaces/train/subject01.leftlight.gif'
image = Image.open(test_image).convert('L')
type(image)

In [ ]:
image

conversão de PIL para numpy

In [ ]:
image_np = np.array(image, "uint8")
cv2_imshow(image_np)
print(image_np.shape)

Numero de pixels

In [ ]:
243 * 320